#### Installing Dependencies

In [ ]:
!sudo apt install octave

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
octave is already the newest version (6.4.0-2).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!pip install oct2py

#### Import Pkgs

In [ ]:
import torch;
import torch.nn as nn;
import torch.nn.functional as F;
from torch.utils.data import Dataset, DataLoader;

import matplotlib.pyplot as plt;
import numpy as np;

from tqdm import tqdm;

In [ ]:
import os;
import os.path;

In [ ]:
import re;

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
print(device);

cuda


In [ ]:
from oct2py import Oct2Py

oc = Oct2Py();

#### Dataset Creation

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, code, healty_end_points, faulty_end_points,oc, index, width):
        self.root_dir = root_dir;
        self.healty_end_points = healty_end_points;
        self.faulty_end_points = faulty_end_points;
        self.oc = oc;
        self.index = index;

        self.width = width;

        self.X = [];
        self.Y = [];

        for healty_end_point in self.healty_end_points:
            for mat_healthy_end_point in os.listdir(os.path.join(self.root_dir, healty_end_point)):
                if mat_healthy_end_point.endswith(".mat") and mat_healthy_end_point.startswith(code):
                    self.X.append(os.path.join(self.root_dir, healty_end_point, mat_healthy_end_point));
                    self.Y.append(0);

        for faulty_end_point in self.faulty_end_points:
            for mat_faulty_end_point in os.listdir(os.path.join(self.root_dir, faulty_end_point)):
                if mat_faulty_end_point.endswith(".mat") and mat_faulty_end_point.startswith(code):
                    self.X.append(os.path.join(self.root_dir, faulty_end_point, mat_faulty_end_point));
                    self.Y.append(1);

    def __len__(self):
        return len(self.X);

    def __getitem__(self, idx):
        data = self.oc.load(self.X[idx]);
        field = list(data.keys())[0];

        x = data[field]['Y']['Data'][0][self.index].flatten();

        w = x.size;

        if(w < self.width):
            x = np.pad(x, (0, self.width - w), 'constant');
        elif(w > self.width):
            x = x[:self.width];

        x = torch.from_numpy(x);
        x = x.to(torch.float32);
        y = torch.tensor(self.Y[idx]);

        x_min = x.min();
        x_max = x.max();
        xx = x

        xx_normal = (xx - x_min) / (x_max - x_min);
        xx_normal = 2 * xx_normal - 1;

        return xx_normal.unsqueeze(dim=0), y;

In [ ]:
def getCode(N, M, F):
    return f"N{N:02}" + "_" + f"M{M:02}" + "_" + f"F{F:02}";

In [ ]:
training_dataset_healthy_end_point = ['K001','K002','K003','K004','K005'];
training_dataset_faulty_end_point = ['KA04','KI04'];

testing_dataset_healthy_end_point = ['K006'];
testing_dataset_faulty_end_point = ['KA30','KI21'];

In [ ]:
N = [9,15];
M = [7,1];
Fa = [10,4];

code = getCode(15,7,10);

In [ ]:
training_dataset = CustomDataset("/content/drive/MyDrive/PaderbornExtracted", code, training_dataset_healthy_end_point, training_dataset_faulty_end_point, oc, 6, 256000);
testing_dataset = CustomDataset("/content/drive/MyDrive/PaderbornExtracted", code, testing_dataset_healthy_end_point, testing_dataset_faulty_end_point, oc, 6, 256000);

In [ ]:
training_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True);
testing_dataloader = DataLoader(testing_dataset, batch_size=32, shuffle=True);

#### Model Creation

In [ ]:
import torch
import torch.nn as nn

class TransposeLayer(nn.Module):
    def __init__(self,dim1,dim2):
        super(TransposeLayer, self).__init__()

        self.dim1 = dim1;
        self.dim2 = dim2;

    def forward(self, x):
        return x.transpose(self.dim1, self.dim2);

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=7, stride=1, downsample=False):
        super(ResidualBlock, self).__init__()
        padding = kernel_size // 2

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.ln1 = nn.LayerNorm(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.ln2 = nn.LayerNorm(out_channels)

        self.downsample = nn.Sequential()
        if downsample or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                TransposeLayer(1,2),
                nn.LayerNorm(out_channels),
                TransposeLayer(1,2)
            )

    def forward(self, x):
        identity = self.downsample(x)

        out = self.conv1(x)
        out = out.transpose(1, 2)
        out = self.ln1(out)
        out = out.transpose(1, 2)
        out = self.relu(out)

        out = self.conv2(out)
        out = out.transpose(1, 2)
        out = self.ln2(out)
        out = out.transpose(1, 2)

        out += identity
        out = self.relu(out)

        return out

class ResNet1D_LN(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet1D_LN, self).__init__()
        self.input_layer = nn.Conv1d(1, 16, kernel_size=7, stride=2, padding=3, bias=False)
        self.ln = nn.LayerNorm(16)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(16, 32, num_blocks=2, downsample=True)
        self.layer2 = self._make_layer(32, 64, num_blocks=2, downsample=True)
        self.layer3 = self._make_layer(64, 64, num_blocks=2, downsample=True)

        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, downsample):
        layers = [ResidualBlock(in_channels, out_channels, downsample=True)]
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.input_layer(x)
        x = x.transpose(1, 2)
        x = self.ln(x)
        x = x.transpose(1, 2)
        x = self.relu(x)
        x = self.pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
model = ResNet1D_LN(num_classes=2)
model = model.to(device);

In [ ]:
print(model);

ResNet1D_LN(
  (input_layer): Conv1d(1, 16, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv1d(16, 32, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
      (ln1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
      (ln2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (downsample): Sequential(
        (0): Conv1d(16, 32, kernel_size=(1,), stride=(1,), bias=False)
        (1): TransposeLayer()
        (2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (3): TransposeLayer()
      )
    )
    (1): ResidualBlock(
      (conv1): Conv1d(32, 32, kernel_size=(7,), stride=(1,), padding=(

In [ ]:
loss = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001);

#### Model Training

In [ ]:
epochs = 75;

In [ ]:
train_loss = [];
test_loss = [];

train_acc = [];
test_acc = [];

for epoch in range(epochs):

    train_loss_per_epoch = [];
    test_loss_per_epcoh = [];

    train_acc_per_epoch = [];
    test_acc_per_epoch = [];

    model.train();
    for x, y in training_dataloader:
        optimizer.zero_grad();
        x = x.to(device);
        y = y.to(device);

        y_pred = model(x);
        loss_value = loss(y_pred, y);

        loss_value.backward();
        optimizer.step();

        train_loss_per_epoch.append(loss_value.item());
        train_acc_per_epoch.append((y_pred.argmax(dim=1) == y).float().mean().item());

    model.eval();
    with torch.no_grad():
        for x, y in testing_dataloader:
            x = x.to(device);
            y = y.to(device);

            y_pred = model(x);
            loss_value = loss(y_pred, y);

            test_loss_per_epcoh.append(loss_value.item());
            test_acc_per_epoch.append((y_pred.argmax(dim=1) == y).float().mean().item());

    train_loss.append(np.mean(train_loss_per_epoch));
    test_loss.append(np.mean(test_loss_per_epcoh));

    train_acc.append(np.mean(train_acc_per_epoch));
    test_acc.append(np.mean(test_acc_per_epoch));

    print(f"Epoch: {epoch+1}/{epochs}, Train Loss: {train_loss[-1]:.4f}, Test Loss: {test_loss[-1]:.4f}, Train Acc: {train_acc[-1]:.4f}, Test Acc: {test_acc[-1]:.4f}");

Epoch: 1/75, Train Loss: 0.6146, Test Loss: 1.1295, Train Acc: 0.7292, Test Acc: 0.3304
Epoch: 2/75, Train Loss: 0.5995, Test Loss: 0.9785, Train Acc: 0.7188, Test Acc: 0.3304
Epoch: 3/75, Train Loss: 0.5618, Test Loss: 0.9476, Train Acc: 0.7188, Test Acc: 0.3326
Epoch: 4/75, Train Loss: 0.5529, Test Loss: 1.0282, Train Acc: 0.7417, Test Acc: 0.3973
Epoch: 5/75, Train Loss: 0.5773, Test Loss: 1.0977, Train Acc: 0.7292, Test Acc: 0.3527
Epoch: 6/75, Train Loss: 0.5350, Test Loss: 1.0863, Train Acc: 0.7438, Test Acc: 0.3348
Epoch: 7/75, Train Loss: 0.5187, Test Loss: 0.9481, Train Acc: 0.7458, Test Acc: 0.3973
Epoch: 8/75, Train Loss: 0.4913, Test Loss: 1.1014, Train Acc: 0.8021, Test Acc: 0.3460
Epoch: 9/75, Train Loss: 0.5182, Test Loss: 0.9652, Train Acc: 0.7854, Test Acc: 0.4330
Epoch: 10/75, Train Loss: 0.4846, Test Loss: 1.1348, Train Acc: 0.7979, Test Acc: 0.3527
Epoch: 11/75, Train Loss: 0.4708, Test Loss: 0.9467, Train Acc: 0.7917, Test Acc: 0.3504
Epoch: 12/75, Train Loss: 0.44

AttributeError: 'NoneType' object has no attribute 'keys'